In [ ]:
!wget https://storage.googleapis.com/tydiqa/v1.0/tydiqa-v1.0-train.jsonl.gz

In [ ]:
ls

In [ ]:
import gzip
import json
import pandas as pd

In [ ]:
jsonfilename = 'tydiqa-v1.0-train.jsonl.gz'

with gzip.open(jsonfilename, 'r') as json_file:
    json_list = list(json_file)

In [ ]:
for json_str in json_list[11:]:
    result = json.loads(json_str)
#     print(f"result: {result}")
#     print(isinstance(result, dict))
    for key in list(result.keys()):
        print(f'{key}: {result[key]}')
        if isinstance(result[key], list):
            print('*' * 60)
            print(key)
            for entry in result[key]:
                if isinstance(entry, dict):
                    print('*' * 60)
                    print(key)
                    print(entry.keys())
                    for key in list(entry.keys()):
                        print(f'{key}: {entry[key]}')
    break


In [ ]:
parsed = []

for json_str in json_list:
    result = json.loads(json_str)
    document_title = result['document_title']
    language = result['language']
    question_text = result['question_text']
    document_plaintext = result['document_plaintext']
    example_id = result['example_id']
    minimal_answers = [x['minimal_answer'] for x in result['annotations']]
    parsed.append({
        'document_title': document_title,
        'language': language,
        'question_text': question_text,
        'document_plaintext': document_plaintext,
        'minimal_answers': minimal_answers,
        'id': example_id,
        })


In [ ]:
df = pd.DataFrame(parsed)
df

In [ ]:
def answer_count(l):
    i = 0
    for x in l:
        if x['plaintext_end_byte'] != -1: i += 1
    return i

df['answer_count'] = df.minimal_answers.apply(answer_count)
df.answer_count.value_counts()

In [ ]:
df = df[df.answer_count != 0].reset_index(drop=True)

In [ ]:
import collections
TextSpan = collections.namedtuple("TextSpan", "byte_positions text")

def byte_slice(text, start, end, errors="replace"):
    # Python 3 encodes text as character sequences, not byte sequences
    # (like Python 2).
    return byte_str(text)[start:end].decode("utf-8", errors=errors)

def byte_str(text):
    return text.encode("utf-8")


def byte_len(text):
    # Python 3 encodes text as character sequences, not byte sequences
    # (like Python 2).
    return len(byte_str(text))

def get_text_span(document_plaintext, span):
    """Returns the text in the example's document in the given span."""
    byte_positions = []
    # `text` is a byte string since `document_plaintext` is also a byte string.
    start = span["plaintext_start_byte"]
    end = span["plaintext_end_byte"]
    text = byte_slice(document_plaintext, start, end)
    for i in range(span["plaintext_start_byte"], span["plaintext_end_byte"]):
        byte_positions.append(i)
#     return TextSpan(byte_positions, text)
    return text

In [ ]:
df.loc[0]

In [ ]:
get_text_span(df.document_plaintext.loc[1], df.minimal_answers.loc[1][0])

In [ ]:
def answer_list(r):
    document_plaintext = r[0]
    minimal_answers = r[1]
    answers = []
    for span in minimal_answers:
        a = get_text_span(document_plaintext, span)
        if a != '': answers.append(a)
    return list(set(answers))

df['answers'] = df[['document_plaintext', 'minimal_answers']].apply(answer_list, axis=1)

In [ ]:
df['answer_count_dedup'] = df.answers.apply(len)
df['answer_count_dedup'].value_counts()

In [ ]:
import re

def get_starts(r):
    answer, text = r[0][0], r[1]
    answer = answer.replace(')', '\)').replace('(', '\(').replace('+', '\+').replace('[', '\[').replace('*', '\*')
    try: 
        return [m.start() for m in re.finditer(answer, text)]
    except: 
        print(answer)

df['starts_a1'] = df[['answers', 'document_plaintext']].apply(get_starts, axis=1)

In [ ]:
get_starts(df[['answers', 'document_plaintext']].loc[16])

In [ ]:
df['byte_starts'] = df.minimal_answers.apply(lambda l: list(set([x['plaintext_start_byte'] for x in l if x['plaintext_start_byte'] != -1])))

In [ ]:
def byte_to_char_offset(b_string, b_offset, encoding='utf8'):
    return len(byte_slice(b_string, 0, b_offset))

def b2c(r):
    b_string = r[0]
    b_offset = r[1][0]
    return byte_to_char_offset(b_string, b_offset, encoding='utf8')

df['starts_a1_frombyte'] = df[['document_plaintext', 'byte_starts']].apply(b2c, axis=1)

In [ ]:
df

In [ ]:
df['answer'] = df['answers'].apply(lambda x: x[0])

In [ ]:
df = df[['id','document_plaintext','question_text', 'answer', 'starts_a1_frombyte', 'language']]
df.columns = ['id', 'context', 'question', 'answer_text', 'answer_start', 'language']
df.head()


In [ ]:
len(df)

In [ ]:
df.language.value_counts()

In [ ]:
df.to_csv('tydiqa_train.csv', index=False)